In [2]:
import torch

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [4]:
torch.manual_seed(42)

In [5]:
df=pd.read_csv('./fmnist_small.csv')

In [6]:
df

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,1,0,0,0,0,0,0,0,0,0,...,69,12,0,0,0,0,0,0,0,0
5996,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5997,8,0,0,0,0,0,0,0,0,0,...,39,47,2,0,0,29,0,0,0,0
5998,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [8]:
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [9]:
X_train = X_train/255
X_test = X_test/255

In [10]:
class CustomDataset(Dataset):
    def __init__(self,features,labels):
        super().__init__()
        self.features= torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels,dtype=torch.long)
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        return self.features[index],self.labels[index]

In [11]:
train_dataset = CustomDataset(X_train,y_train)

In [12]:
len(train_dataset)

4800

In [13]:
train_dataset[0]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [14]:
test_dataset = CustomDataset(X_test,y_test)

In [15]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False)

In [16]:
class MyNN(nn.Module):
    def __init__(self,num_features):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,10)
        )
    def forward(self,x):
        return self.model(x)

In [17]:
learning_rate = 0.1
epochs = 100

In [18]:
model = MyNN(X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [19]:
for epoch in range(epochs):
    total_epoch_loss = 0
    for batch_features,batch_labels in train_loader:
        outputs = model(batch_features)
        
        loss = criterion(outputs,batch_labels)
        
        total_epoch_loss+=loss.item()
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
    print(total_epoch_loss/len(train_loader))

1.3216368559996288
0.779336535135905
0.6427524556716283
0.5751657243569692
0.5278772535920143
0.49531099955240887
0.4619268517692884
0.4355265100300312
0.41894517570734024
0.3993094889819622
0.38615925490856173
0.3742155360182126
0.34714958479007085
0.3478396432598432
0.31352100044488906
0.31205256044864654
0.29320213382442795
0.2866890056431293
0.2748382142186165
0.26294573038816454
0.25803839137156803
0.24635562914113204
0.23844928378860156
0.22297091687719028
0.22178592306872208
0.20807982956369717
0.2153981725871563
0.2099522104486823
0.1882164634640018
0.1902953559036056
0.18715596643586954
0.19899731529255707
0.17918923144539198
0.17041035249829292
0.1626656906803449
0.16609758401910463
0.16023195638010898
0.14909806532164416
0.14557932056486605
0.13712968522527566
0.13066986605525016
0.13339852248008052
0.12634767746552825
0.1205450955654184
0.1110396111321946
0.12403606501718362
0.1104420518875122
0.11793826536585887
0.10102571475630005
0.11747935121878982
0.09050587536767125
0

In [20]:
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [21]:
total = 0
correct = 0
with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        outputs = model(batch_features)
        _,predicted = torch.max(outputs,1)
        total+=batch_labels.shape[0]
        correct+=(predicted==batch_labels).sum().item()
        
print(correct/total)

0.8275
